In [320]:
import pandas as pd
df = pd.read_csv('projet.csv')

In [321]:
import numpy as np

# créer une nouvelle colonne de titre
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)


# remplacer les titres rares par des titres plus courants
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',
           'Don': 'Mr', 'Mme': 'Mrs', 'Jonkheer': 'Mr', 'Lady': 'Mrs',
           'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}
df.replace({'Title': mapping}, inplace=True)

df['Title'].value_counts()


title_ages = dict(df.groupby('Title')['Age'].median())

# créer une colonne des âges moyens
df['age_med'] = df['Title'].apply(lambda x: title_ages[x])

df['Age'].fillna(df['age_med'], inplace=True, )
del df['age_med']

In [322]:
#df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'].fillna(method='backfill', inplace=True)

# créer la colonne Family_Size (Parch + SibSp)
df['Family_Size'] = df['Parch'] + df['SibSp']
df['Sex'] = [0 if x=="male" else 1 for x in df["Embarked"]]

print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    1  35.0      0      0   

             Ticket     Fare Cabin Embarked Title  Family_Size  
0         A/5 21171   7.2500   NaN        S    Mr            1  
1          PC 17599  71.2833   C85        C   Mrs            1  
2  STON/O2. 3101282   7.9250   NaN        S  Miss            0  
3            113803  53.1000  C123  

In [323]:
data = df.drop(['Survived' , 'PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , axis = 1)
#Convertir une variable en plusieurs variables
dataset = pd.get_dummies(data , columns = ['Embarked', 'Title']  , drop_first = False) 
properties = list(dataset.columns.values)
X = dataset[properties]
y = df['Survived']
print(X)

     Pclass  Sex   Age  SibSp  Parch     Fare  Family_Size  Embarked_C  \
0         3    1  22.0      1      0   7.2500            1           0   
1         1    1  38.0      1      0  71.2833            1           1   
2         3    1  26.0      0      0   7.9250            0           0   
3         1    1  35.0      1      0  53.1000            1           0   
4         3    1  35.0      0      0   8.0500            0           0   
..      ...  ...   ...    ...    ...      ...          ...         ...   
886       2    1  27.0      0      0  13.0000            0           0   
887       1    1  19.0      0      0  30.0000            0           0   
888       3    1  21.0      1      2  23.4500            3           0   
889       1    1  26.0      0      0  30.0000            0           1   
890       3    1  32.0      0      0   7.7500            0           0   

     Embarked_Q  Embarked_S  Title_Dr  Title_Master  Title_Miss  Title_Mr  \
0             0           1       

In [324]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 ,random_state = 0)

from sklearn.preprocessing import StandardScaler
# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [325]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(X.shape[1],)),
	keras.layers.Dense(18, input_shape=(X.shape[1],) , activation=tf.nn.relu),
    keras.layers.Dense(1,  activation=tf.nn.sigmoid),
])

print(model.summary())

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 18)                306       
_________________________________________________________________
dense_104 (Dense)            (None, 1)                 19        
Total params: 325
Trainable params: 325
Non-trainable params: 0
_________________________________________________________________
None


In [326]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=10, batch_size=1 , verbose=1)
test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/10
596/596 [==============================] - 4s 5ms/step - loss: 0.6256 - accuracy: 0.6846
Epoch 2/10
596/596 [==============================] - 3s 5ms/step - loss: 0.4624 - accuracy: 0.8205
Epoch 3/10
596/596 [==============================] - 3s 5ms/step - loss: 0.4221 - accuracy: 0.8372: 0s - loss: 0.4310 - accuracy
Epoch 4/10
596/596 [==============================] - 3s 5ms/step - loss: 0.4071 - accuracy: 0.8372
Epoch 5/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3985 - accuracy: 0.8406: 1s - ETA: 0s - loss: 0.4065 - 
Epoch 6/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3942 - accuracy: 0.8406
Epoch 7/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3896 - accuracy: 0.8473
Epoch 8/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3852 - accuracy: 0.8456: 3s - loss: 0.3 -
Epoch 9/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3839 - accuracy: 0.8423
Epoch 10/10
378/596 

In [287]:
import pandas as pd
y_pred = model.predict(X_test)

In [276]:
import pandas as pd
y_pred = model.predict(X_test)
y_pred[0:5]

array([[0.17754692],
       [0.09806672],
       [0.00599256],
       [0.98778135],
       [0.88981223]], dtype=float32)

In [278]:
y_test.head(5)

495    0
648    0
278    0
31     1
255    1
Name: Survived, dtype: int64